In [1]:
from tqdm import tqdm
from glob import glob
import tifffile
import numpy as np
import os
from EmbedSeg.utils.preprocess_data import extract_data, split_train_val, split_train_test, split_train_crops, get_data_properties
from EmbedSeg.utils.generate_crops import *
import json

### Download Data

In [2]:
data_dir = '../../../data'
project_name = 'bbbc010-2012'

Ideally, <b>*.tif</b>-type images and the corresponding masks should be respectively present under <b>images</b> and <b>masks</b>, under directories <b>train</b>, <b>val</b> and <b>test</b>, which can be present at any location on your workstation, pointed to by the variable <i>data_dir</i>. (In order to prepare such instance masks, one could use the Fiji plugin <b>Labkit</b> as detailed <b>[here](https://github.com/juglab/EmbedSeg/wiki/Use-Labkit-to-prepare-instance-masks)</b>). The following would be the desired structure as to how data should be present. 

<img src="../../../static/png/01_dir_structure.png" width="100"/>

If you already have your data available in the above style, please skip to the <b><a href="#center">third</a></b> section of this notebook, where you specify the kind of center to which constitutive pixels of an object should embed. 
Since for the <b> bbbc010-2012</b> we do not have the data in this format yet, we firstly download the data from an external url in the following cells, next we split this data to create our `train`, `val` and `test` directories. 
     

The images and corresponding masks are downloaded from an external url, specified by `zip_url` to the path specified by the variables `data_dir` and `project_name`. The following structure is generated after executing the `extract_data`, `split_train_test` and `split_train_val` methods below:

<img src="../../../static/png/02_bbbc010-2012.png" width="400"/>

In [3]:
extract_data(
    zip_url = 'https://github.com/juglab/EmbedSeg/releases/download/v0.1.0/bbbc010-2012.zip',
    data_dir = data_dir,
    project_name = project_name,
)

Downloaded data as ../../../data/bbbc010-2012.zip
Unzipped data to ../../../data/bbbc010-2012/download/


### Split Data into `train`, `val` \& `test`

Since the `train`-`test` data partition doesn't exist by itself in the original data, we can execute the following cell to reserve some data as evaluation or test data. Here, we reserve 50 % of the available data for evaluation, as is usually done in literature, with regards to the `bbbc010-2012` dataset.

In [4]:
split_train_test(
    data_dir = data_dir,
    project_name = project_name, 
    train_test_name = 'train',
    subset = 0.5)

Created new directory : ../../../data/bbbc010-2012/download/test/images
Created new directory : ../../../data/bbbc010-2012/download/test/masks
Train-Test Images/Masks saved at ../../../data/bbbc010-2012/download


For this dataset, instead of reserving a small fraction of the train dataset for validation at this stage, we first crop the images and masks in the subsequent code cells, and <b><a href= "split_val">later</a></b> reserve some of the generated crops for the purposes of validation. We notice that such a strategy allows better results for `bbbc010-2012` during prediction (because of a small dataset size). Running the next cell simply copies the train and test images and masks to the `$data_dir/$project_name/train/.` and `$data_dir/$project_name/test/.` respectively.

In [5]:
split_train_val(
    data_dir = data_dir,
    project_name = project_name, 
    train_val_name = 'train',
    subset = 0.0)

Created new directory : ../../../data/bbbc010-2012/train/images/
Created new directory : ../../../data/bbbc010-2012/train/masks/
Created new directory : ../../../data/bbbc010-2012/val/images/
Created new directory : ../../../data/bbbc010-2012/val/masks/
Created new directory : ../../../data/bbbc010-2012/test/images/
Created new directory : ../../../data/bbbc010-2012/test/masks/
Train-Val-Test Images/Masks copied to ../../../data/bbbc010-2012


### Specify desired centre location for spatial embedding of pixels
<a id='center'></a>

Interior pixels of an object instance can either be embedded at the `medoid`, the `approximate-medoid` or the `centroid`. 

In [6]:
center = 'medoid'  # 'medoid', 'approximate-medoid', 'centroid'
try:
    assert center in {'medoid', 'approximate-medoid', 'centroid'}
    print("Spatial Embedding Location chosen as : {}".format(center))
except AssertionError as e:
    e.args += ('Please specify center as one of : {"medoid", "approximate-medoid", "centroid"}', 42)
    raise



Spatial Embedding Location chosen as : medoid


### Calculate some dataset specific properties

In the next cell, we will calculate properties of the data such as `min_object_size`, `foreground_weight` etc. <br>
We will also specify some properties, for example,  
* set `data_properties_dir['one_hot'] = True` in case the instances are encoded in a one-hot style. 
* set `data_properties_dir['data_type']='16-bit'` if the images are of datatype `unsigned 16 bit` and 
    `data_properties_dir['data_type']='8-bit'` if the images are of datatype `unsigned 8 bit`.

Lastly, we will save the dictionary `data_properties_dir` in a json file, which we will access in the `02-train` and `03-predict` notebooks.

In [7]:
one_hot = True
data_properties_dir = get_data_properties(data_dir, project_name, train_val_name=['train'], 
                                          test_name=['test'], mode='2d', one_hot=one_hot)

data_properties_dir['data_type']='16-bit'

with open('data_properties.json', 'w') as outfile:
    json.dump(data_properties_dir, outfile)
    print("Dataset properies of the `{}` dataset is saved to `data_properties.json`".format(project_name))

 14%|█▍        | 7/50 [00:00<00:00, 65.29it/s]

Foreground weight of the `bbbc010-2012` dataset set equal to 10.000


 50%|█████     | 25/50 [00:00<00:00, 242.34it/s]

Minimum object size of the `bbbc010-2012` dataset is equal to 491


100%|██████████| 50/50 [00:00<00:00, 250.30it/s]

Maximum evaluation image size of the `bbbc010-2012` dataset set equal to (1024, 1024)
Dataset properies of the `bbbc010-2012` dataset is saved to `data_properties.json`


### Specify cropping configuration parameters

Images and the corresponding masks are cropped into patches centred around an object instance, which are pre-saved prior to initiating the training. Note that the cropped images, masks and center-images would be saved at the path specified by `crops_dir` (The parameter `crops_dir` is set to ```./crops``` by default, which creates a directory at the same location as this notebook). Here, `data_subset` defines the directory which is processed. Since we only have `train` images and masks at `$data_dir/$project_name/train`, hence we set `data_subset=train`.

In [8]:
crops_dir = './crops'
data_subset = 'train' 
crop_size = 256

### Generate Crops



<div class="alert alert-block alert-warning"> 
    The cropped images and masks are saved at the same-location as the example notebooks. <br>
    Generating the crops might take a little while!
</div>

In [9]:
image_dir = os.path.join(data_dir, project_name, data_subset, 'images')
instance_dir = os.path.join(data_dir, project_name, data_subset, 'masks')
image_names = sorted(glob(os.path.join(image_dir, '*.tif'))) 
instance_names = sorted(glob(os.path.join(instance_dir, '*.tif')))  
for i in tqdm(np.arange(len(image_names))):
    if one_hot:
        process_one_hot(image_names[i], instance_names[i], os.path.join(crops_dir, project_name), data_subset, crop_size, center, one_hot = one_hot)
    else:
        process(image_names[i], instance_names[i], os.path.join(crops_dir, project_name), data_subset, crop_size, center, one_hot=one_hot)
print("Cropping of images, instances and centre_images for data_subset = `{}` done!".format(data_subset))

100%|██████████| 50/50 [00:25<00:00,  1.93it/s]

Cropping of images, instances and centre_images for data_subset = `train` done!


### Move a fraction of the generated crops for validation purposes

Here we reserve a small fraction (15 \% by default) of the images, masks and center-images crops for the purpose of validation. 
<a id="later_val">

In [10]:
split_train_crops(project_name = project_name, center = center, crops_dir = crops_dir, subset = 0.15)

Created new directory : ./crops/bbbc010-2012/val/images/
Created new directory : ./crops/bbbc010-2012/val/masks/
Created new directory : ./crops/bbbc010-2012/val/center-medoid/
Val Images/Masks/Center-medoid-image crops saved at ./crops/bbbc010-2012/val
